In [1]:
import pandas as pd

# 1. Load CSV files
admissions = pd.read_csv('/root/DATA/admissions.csv')
bloodculture = pd.read_csv('/root/DATA/bloodculture.csv')
gcs_hourly = pd.read_csv('/root/DATA/gcs_hourly.csv')
icustays = pd.read_csv('/root/DATA/icustays.csv')
labs_hourly = pd.read_csv('/root/DATA/labs_hourly.csv')
pt_icu_outcome = pd.read_csv('/root/DATA/pt_icu_outcome.csv')

# 2. Filter icustays to retain records where Length of Stay (LOS) is greater than 1.0 day
icustays_filtered = icustays[icustays['los'] > 1.0]

# 3. Keep only the first ICU admission for each patient (grouped by subject_id)
icustays_first_admission = (
    icustays_filtered.sort_values('intime').groupby('subject_id').first().reset_index()
)
print(f"Shape after filtering first ICU admission records: {icustays_first_admission.shape}")

# 4. Filter bloodculture, labs_hourly, and gcs_hourly data to keep records within the first 24 hours (hr between 0 and 24)
bloodculture_24h = bloodculture[bloodculture['hr'].between(0, 24)]
labs_hourly_24h = labs_hourly[labs_hourly['hr'].between(0, 24)]
gcs_hourly_24h = gcs_hourly[gcs_hourly['hr'].between(0, 24)]

print(f"Filtered bloodculture shape: {bloodculture_24h.shape}")
print(f"Filtered labs_hourly shape: {labs_hourly_24h.shape}")
print(f"Filtered gcs_hourly shape: {gcs_hourly_24h.shape}")

# 5. Merge with admissions to retain only patients with a first ICU admission
merged_data = pd.merge(
    icustays_first_admission, admissions, on='hadm_id', how='inner'
)
print(f"Shape after merging icustays and admissions: {merged_data.shape}")

# 6. Merge with bloodculture data
merged_data = pd.merge(
    merged_data, bloodculture_24h, on='icustay_id', how='inner'
)
print(f"Shape after merging bloodculture: {merged_data.shape}")

# 7. Merge with gcs_hourly data
merged_data = pd.merge(
    merged_data, gcs_hourly_24h, on='icustay_id', how='inner'
)
print(f"Shape after merging gcs_hourly: {merged_data.shape}")

# 8. Merge with labs_hourly data
merged_data = pd.merge(
    merged_data, labs_hourly_24h, on='icustay_id', how='inner'
)
print(f"Shape after merging labs_hourly: {merged_data.shape}")

# 9. Merge with pt_icu_outcome data
merged_data = pd.merge(
    merged_data, pt_icu_outcome, on='icustay_id', how='inner'
)
print(f"Shape after merging pt_icu_outcome: {merged_data.shape}")

# 10. Check for NaN values and print a summary
print("Summary of NaN values after merging:")
print(merged_data.isna().sum())

# 11. Export the merged data to a CSV file
output_path = '/root/DATA/filtered_merged_data.csv'
merged_data.to_csv(output_path, index=False)
print(f"The processed data has been exported to {output_path}")


Shape after filtering first ICU admission records: (37441, 12)
Filtered bloodculture shape: (127913, 10)
Filtered labs_hourly shape: (262866, 22)
Filtered gcs_hourly shape: (400345, 7)
Shape after merging icustays and admissions: (37441, 30)
Shape after merging bloodculture: (75930, 39)
Shape after merging gcs_hourly: (561261, 45)
Shape after merging labs_hourly: (2938748, 66)
Shape after merging pt_icu_outcome: (2938748, 82)
Summary of NaN values after merging:
subject_id_x                    0
row_id_x                        0
hadm_id_x                       0
icustay_id                      0
dbsource                        0
                           ...   
icu_expire_flag                 0
hospital_expire_flag_y     770581
dod                       1400109
expire_flag                     0
ttd_days                  1400109
Length: 82, dtype: int64
The processed data has been exported to /root/DATA/filtered_merged_data.csv
